# Linear Models for Regression

One of the oldest and most widely used models are linear regression models. Linear models for regression have been a staple in statistics and engineering long before the invention of the terms Computer Science and Machine Learning TODO cite. In fact, one of the most commonly used approaches goes back to Gauss himself, and you might have seen some variant of it in high school. Linear models for regression also serve as the basis for the linear models for classification that we will see in section TODO.

The principle of linear models is illustrated in Figure TODO which shows a linear regression in one dimension, in our terminology we have one feature, shown on the x-axis, and our target is shown on the y-axis.
We now try to find a linear relationship between the two, in other words, we want to find a straight line that is as close as possible to all of the points.
For a single feature, you can write the prediction given by the line as
$$ \hat{y} = a x + b $$
where $a$ and $b$ are fixed numbers, and $x$ is the value of our input feature. Here, $a$ is the slope of the line, and $b$ is known as the intercept or offset. It is the place where the fitted line crossed the y-axis.
For more than one feature, the prediction takes the form
$$ \hat{y} = w_1  x_1 + w_2  x_2 + ...  + w_p x_p + b $$
where $x_1$ to $x_p$ are the features (i.e. we have p features) and $w_1$ to $w_p$ are the coefficients or slope for each of these features.
For more than one features, visualizing the prediction is much trickier, so we'll skip that for now.

```{admonition} Mathematical details
Often the above equation is written as as an inner product between two vectors $w \in \mathbb{R}^p$ and $x\in \mathbb{R}^p$:
$$\hat{y} = w^T \mathbf{x} + b = \sum_{i=1}^p w_i x_i +b$$
As a side note, linear models are named such because they are linear in $w$, not because they are linear in $x$. In fact, we can replace any $x_i$ by an arbitrary fixed function of $x$, say $sin(x_i)$ or $x_i * x_j$, and it would still be considered a linear model. This is one of the powers of this kind of model, and we'll discuss it in more depth in chapter TODO feature engineering. 
```

All linear models for regression share the same formular for prediction shown in TODO. However, how they differ is in how they learn the coefficients $w$ and $b$ from the training data. Here, $w$ and $b$ are *parameters* of the model (in the statistical sense), i.e. they are the parts of the model that are estimated from the training data. Nearly all of the methods for learning linear models are formulated as an optimization problem, where the goal is to find the optimum of some mathematical formula, which is known as objective function. We will provide the objective functions (and therefore the optimizatio problem) that give rise to all the models we will discuss. If you are not familiar with optimization, you can skip these parts as we will also provide some more intuitive explanation of each method.

## Ordinary Least Squares
The oldest and probably the most widely use method to learn the parameters $w$ and $b$ is ordinary least squares (OLS). This is what people usually refer to when they say "linear regression" and it's what's implemented in scikit-learn's ``LinearRegression`` class. The idea behind OLS is to compute the *least squares* solution, which means find the parameters $w$ and $b$ so that the prediction error on the training set has the smallest possible squared norm.
This is a classical and well-understood method. However, it requires the training data to be somewhat well-behaved. If there are more features than there are samples, or if there is a linear relationship between the features, then the method can be unstable and will often reduce non-sensical results.
TODO image

```{admonition} Mathematical details
The objective function of ordinary least squares for linear regression is given by
$$\min_{w \in \mathbb{R}^p, b\in\mathbb{R}} \sum_{i=1}^n (w^T\mathbf{x}_i + b - y_i)^2$$
Here, $x_i, i=1,...,n $ are the training vectors and $y_i i=1,...,n$ TODO indexing unlike above?! are the corresponding targets.
The formula on the right computes the prediction error for each training sample, squares it, and then sums over all the samples.
The outcome is a number that measures the euclidean length (aka squared norm) of the error $\hat{y}_i - y_i$. We want to find a vector $w$ and a number $b$ such that this error is minimized.
If the data matrix X has full column rank, then this is a convex optimization problem, and we can easily compute the optimum with a number of standard techniques.
If the data matrix X does not have full column rank, for example if there is more feature than samples, then there is no unique optimum; in fact there will be infinitely many solutions that lead to zero error on the training set.
In this case, the optimization problem is called *ill conditioned* as no unique solution exists. In this case we can add additional criteria that guarantee a well-behaved solution TODO ref, though
they are not always implemented in statistical software.
```

Before we look at examples of using ``LinearRegression`` let's look at an alternative that is particularly popular for predictive modelling, *Ridge Regression*

## Ridge Regression
Ridge regression is an extension to OLS in which we add another demand to our solution. Not only do we want our model to fit the training data well, we also want it to be simple.
We already discussed in chapter TODO, that we prefer simple solutions, as they will generalize more readily to new data. TODO this needs work.
We can include this directly in our modeling process-however, we have to formalize what we mean by "simple". In Ridge regression, simple is encoded as having small slope in all directions.
In other words, we want our coefficient vectors $w$ to be as close as possible to 0 (in fact the restriction is on $w^2$). So now we have to goals: we want a simple model (i.e. one with small coefficients) that also fits the training data well. To allow optimizing both, we need to introduce a parameter, often called $\alpha$ (or sometimes $\lambda$) that trades off between the two goals. Now $\alpha$ is a hyper-parameter that we need to tune or pick, say using `GridSearchCV`. Setting $alpha=0$ will disregard the simplicity demand, and is equivalent to using OLS. Setting $\alpha=\infty$ will basically disregard the data fitting demand, and will produce a coefficient vector of all zeros, which is the most "simple" solution according to the least squares measure used in Ridge regression. The $\alpha$ parameter is known as *regularization parameter* or *penalty term*, as it regularizes or penalizes overly complex solutions $w$.

```{admonition} Mathematical details
The objective function of Ridge regression is very similar to OLS; it only has the addition of the penalty term $\alpha ||w||^2$. Notice that the intercept $b$ is not penalized. TODO explain?
$$ \min_{w \in \mathbb{R}^p, b\in\mathbb{R}} \sum_{i=1}^n (w^T\mathbf{x}_i + b - y_i)^2 + \alpha ||w||^2 $$
The first term in the objective function is also known as the data fitting term, while the second term, the penalty term, is completely independent of the data.
For $\alpha>0$, this objective function is always strongly convex, which leads to simple and robust optimization methods.
```

The benefit of Ridge regression is that it is much more stable than linear regression with OLS. If $\alpha>0$, it always has a unique solution, no matter how badly behaved your data is.
From a statistical perspective Ridge regression has the disadvantage that it is not an unbiased estimator: we explicitly biased our model to make the coefficients small, so predictions are biased towards zero. In predictive modeling, this is rarely an issue, however, you should keep it in mind.

The presence of a hyper-parameter could be seen either as an advantage or a disadvantage: one the one hand, it allows us to control the model fitting more closely, on the other it requires us to use some method to adjust the hyper-parameter, which makes the whole modeling process more complicated.


```{admonition} Mathematical details

## (regularized) Empirical Risk Minimization
The objective function shown in TODO is a typical example of a mathematical framework that's widely used in machine learning,
known as regularized empirical risk minimization.
As mentioned in TODO we didn't mention it yet, the goal in supervised learning is usually to find a function $f$ from a fixed familiy of fuctions $F$ that minimizes

$$\min_{f\in F} \mathbb{E}_{(x, y) \propto p} \ell(f(x), y)$$
for some loss function $\ell$. In other words, we want to minimize the expected loss between the prediction made by $f$ and the true outcome $y$,
when sampling $x$ and $y$ from the data distribution. However, in practice, it's usually impossible to access the data distribution $p$, and instead,
we have an i.i.d. sample from it, the training data.

The idea behind empirical risk minimization is that if we minimize the training set error, and the function $f$ is simple, then the error on new samples will also be small.
In other words, we replace the expectation in formula TODO by the empirical estimate on the training set:
$$\min_{f\in F} \sum_{i=i}^n\ell(f(x_i), y_i)$$
This is now a problem we can solve, and it is exactly the form of the objective found in OLS in equation TODO, where $F$ is the family of all linear models, as parametrized by $w$ and $b$, and $\ell$ is the squared loss.
If we take this approach, it's possible to proof theorems of the form TODO

In other words, we are trying to ensure a small error on the training set, and if we achive that, we can show that the test set error will also be small, if $f$ is sufficiently simple.
We could now try to optimize the bound in Equation TODO more directly, by optimizing both the training set error and encouraging the model to be simple.
This leads to *regularized empirical risk minimization* in which we add an additional *regularization term* $R$:
$$ \min_{f \in F} \sum_{i=1}^n L(f(\mathbf{x}_i), y_i) + \alpha R(f) $$
This is the approach taken by ridge regression, where $R$ is the squared norm of the coefficients.
Maybe somewhat surprisingly, equation TODO summarizes most of supervised machine learning, and nearly every supervised algorithm in this book follows this idea.
Linear models, support vector machines, neural networks and gradient boosting can all be formulated in this way, using different choices of the family of functions F, the loss $\ell$ and the regularizer $R$.
It is also the basis of nearly all theoretical analysis of machine learning algorithms. Interestingly, it is now thought to not adequately explain the generalization ability of the very large neural networks used today, as they are not "simple" in any way we can quantify easily. This has led to a large body of interesting work investigating new theoretical underpinning for deep neural networks TODO reference.
Please keep in mind that this is a very birds-eye view, as we are much more concerned with practical issues in this book.
Theoretical machine learning is a fascinating topic, and you can learn much, much more in the amazing book TODO.
```




### Coefficient of determination
To evaluate model fit for regression, scikit-learn by default uses the coefficient of determination, also known as $R^2$, which is defined as follows:
$$ R^2(y, \hat{y}) = 1 - \frac{\sum_{i=0}^{n - 1} (y_i - \hat{y}_i)^2}{\sum_{i=0}^{n - 1} (y_i - \bar{y})^2} $$
where $\bar{y}$ is the mean response over the training set:
$$ \bar{y} =  \frac{1}{n} \sum_{i=0}^{n - 1} y_i$$
The intuition of the measure it that it looks at the relative average distance between prediction and ground truth.

There are several definitions of this metric, but the one given in Equation TODO works for any model, and is what ``score`` will return for any regressor.
The benefit of this metric is that its scale is easily interpretable: a score of 1 means perfect prediction, while a score of 0 means constant or entirely random predictions.
Note that with this definition, the $R^2$ score can be negative, which basically means that predictions are worse than just predicting the mean.
A disadvantage of this score is that it depends on the variance of the data it is applied to, which has several peculiar implications.
In particular, adding a data point with an extreme true response value will improve the overall score. Also, the mean is estimated independently when scoring the test set,
which can lead to artifacts if the test set is small. In particular, when computing the score on a single data point, it is undefined.
TODO reference Max Kuhn.

An alternative metric that has very different properties, is the root mean squared error (RMSE), which we'll discuss in more detail in Chapter TODO.

## Ames Housing Dataset

![:scale 80%](images/ames_housing_scatter.png)
.tiny[
```python
print(X.shape)
print(y.shape)
```
```
(2195, 79)
(2195,)
```
]


Ok after all this pretty abstract talk, let's make this
concrete. Let's do some regression on the boston housing
dataset. After the last homework you're hopefully familiar
with it. The idea is to predict prices of property in the
boston area in different neighborhoods. This is a dataset
from the 70s I think, so everything is pretty cheap. Most of
the features you can see are continuous, with the exception
of the charlston river variable which says whether the
neighborhood is on the river.

Keep in mind that this data lives in a 13 dimensional space
and these univariate plots only look at 13 different
projections of the data, and can't capture any of the
interactions.

But still we can see that the price clearly depends on some
of these variables. It's also pretty clear that the
dependency is non-linear for some of the variables. We'll
still start with a linear model, because its a very simple
class of models, and I'd always star approaching any model
from the simplest baseline. In this case it's linear
regression. We're having 506 samples and 13 features. We
have much more samples than features. Linear regression
should work just fine. Also it's a tiny dataset, so
basically anything we'll try will run instantaneously, which
is also good to keep in mind.

Another thing that you can see in this graph is that the
features have very different scales. Here's a box plot that
shows that even more clearly.

![:scale 100%](images/ames_scaling.png)



That's something that will trip up the distance based models
models we talked about last time, as well as the linear
models we're talking about today.  For the penalized models
the different scales mean that different features are
penalized differently, which you usually want to avoid.
Usually there is no particular semantics attached to the
fact that one feature has different magnitutes than another.
We could measure something in inches instead of miles, and
that would change the outcome of the model. That's certainly
not something we want. A good idea is to scale the data to
get rid of this effect. We'll talk about that and other
preprocessing methods in-depth on Wednesday next week. Today
I'm mostly gonna ignore this. But let's get started with
Linear Regression

## Preprocessing
```python
cat_preprocessing = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='NA'),
    OneHotEncoder(handle_unknown='ignore'))

cont_preprocessing = make_pipeline(
    SimpleImputer(),
    StandardScaler())

preprocess = make_column_transformer(
    (cat_preprocessing, make_column_selector(dtype_include='object')),
    remainder=cont_preprocessing)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)
cross_val_score(
    make_pipeline(preprocess, LinearRegression()),
    X_train, y_train, cv=5)
```
```
array([0.928, 0.927, 0.932, 0.898, 0.884])
```

## Skewed targets
.left-column[
```python
y_train.hist(bins='auto')
```
![:scale 100%](images/ames_housing_price_hist.png)

]
.right-column[
```python
np.log(y_train).hist(bins='auto')
```
![:scale 100%](images/ames_housing_price_hist_log.png)

]

```python
cross_val_score(make_pipeline(preprocess, LinearRegression()),
                X_train, y_train, cv=5)
```
```
array([0.928, 0.927, 0.932, 0.898, 0.884])
```

```python
from sklearn.compose import TransformedTargetRegressor
log_regressor = TransformedTargetRegressor(
    LinearRegression(), func=np.log, inverse_func=np.exp)
cross_val_score(make_pipeline(preprocess, log_regressor),
                X_train, y_train, cv=5)
```
```
array([0.95 , 0.943, 0.941, 0.913, 0.922])

```

## Linear Regression vs Ridge
```python
log_regressor = TransformedTargetRegressor(
    LinearRegression(), func=np.log, inverse_func=np.exp)
cross_val_score(make_pipeline(preprocess, log_regressor),
                X_train, y_train, cv=5)

```
```
array([0.95 , 0.943, 0.941, 0.913, 0.922])
```
```python
log_ridge = TransformedTargetRegressor(
    Ridge(), func=np.log, inverse_func=np.exp)
cross_val_score(make_pipeline(preprocess, log_ridge),
                X_train, y_train, cv=5)

```
```
array([0.948, 0.95 , 0.941, 0.915, 0.931])

```


Let’s look at two simple models. Linear regression and Ridge
regression. What I've done is I’ve split the data into
training and test set and used 10 fold cross-validation to
evaluate them. Here I use cross_val_score together with the
model, the training data, training labels, and 10 fold
cross-validation. This will return 10 scores and I'm going
to compute the mean of them. I'm doing this for both linear
regression and Ridge regression. Here is ridge regression
uses a default value of alpha of 1. Here these two scores
are quite similar.

.smallest[
```python
pipe = Pipeline([('preprocessing', preprocess), ('ridge', log_ridge)])
param_grid = {'ridge__regressor__alpha': np.logspace(-3, 3, 13)}

grid = GridSearchCV(pipe, param_grid, cv=RepeatedKFold(10, 5),
                    return_train_score=True)
```
```
{'ridge__regressor__alpha': array([ 0.001,  0.003, 0.01, 0.032, 0.1, 0.316, 1., 3.162,
           10., 31.623, 100., 316.228, 1000.])}
```

```python
grid.fit(X_train, y_train)
```
]
.center[
![:scale 50%](images/ridge_alpha_search.png)
]



Coming back to the ridge regression we used the standard
alpha of one which is a reasonable default, but by no means,
this is guaranteed to make sense in this particular problem.
Here I’ve done the grid search. As we talked about on
Monday, I defined a parameter grid where the key is the
parameter I want to search (alpha in Ridge) and the
parameters I want to try. For regularization parameters,
like alpha, it’s usually good to do them on the logarithmic
grid. I do a relatively fine grid here with 13 different
points mostly because I wanted to have a nice plot. In
reality, I would use a three or six or something like that.
I’ve instantiated GridSearchCV with Ridge(), the parameter
grid and do 10 fold cross-validation and then I called
grid.fit. I’ve reported the mean training accuracy and mean
test accuracy over 10 cross-validation folds for each of the
parameter settings. Okay, you can see a couple things here.
A) There's a lot of uncertainty B) The training set is
always better than the test set. C) The most important thing
that you can see here is that regularization didn't help.
Making alpha as small as possible is the best. What I'm
going to do next is I'm going to modify this dataset a
little bit so that we can see the effect of the
regularization. I’m going to modifying by using a polynomial
expansion, again we're going to talk about a little bit more
on Wednesday.

.padding-top[
.left-column[
![:scale 100%](images/ridge_alpha_search.png)
]
.right-column[
![:scale 100%](images/ridge_alpha_search_cv_runs.png)
]
]

## Triazine Dataset

```python
triazines = fetch_openml('triazines')
triazines.data.shape
```
```
(186, 60)
```
```python
pd.Series(triazines.target).hist()
```
.center[
![:scale 40%](images/triazine_bar.png)
]

```python
X_train, X_test, y_train, y_test = train_test_split(triazines.data, triazines.target, random_state=0)

cross_val_score(LinearRegression(), X_train, y_train, cv=5)
```
```
array([-4.749e+24, -9.224e+24, -7.317e+23, -2.318e+23, -2.733e+22])
```
```python
cross_val_score(Ridge(), X_train, y_train, cv=5)
```
```
array([0.263, 0.455, 0.024, 0.23 , 0.036])
```

```python
param_grid = {'alpha': np.logspace(-3, 3, 13)}

grid = GridSearchCV(Ridge(), param_grid, cv=RepeatedKFold(10, 5),
                    return_train_score=True)
grid.fit(X_train, y_train)
```
.center[
![:scale 40%](images/ridge_alpha_triazine.png)
]

## Plotting coefficient values (LR)


```python
lr = LinearRegression().fit(X_train, y_train)
plt.scatter(range(X_train.shape[1]), lr.coef_,
            c=np.sign(lr.coef_), cmap='bwr_r')

```
.center[
![:scale 55%](images/lr_coefficients_large.png)
]



In the previous slide, linear regression did nearly as well
as the Ridge Regression with the default parameters but
let's look at the coefficients of a linear model. These are
the coefficients of the linear model trained on the
polynomial futures. I plot them adding a color to represent
positive and negative. The magnitude here is 1 and then 13
zeros. We have two features, one is like, probably more than
trillions times two and then there's another one that's very
negative. What I conclude from this is, these 2 features are
very highly correlated. The model makes both of them really,
really big and then they cancel each other out. This is not
a very nice model, because it relates to American stability
and also it tells me that these 2 features are like
extremely important, but they might not be important at all.
Like the other features might be more important, but they're
nullified by this cancellation effect. They (0.2 and -0.8)
need to cancel each other out because the predictions are
reasonable and all the houses only cost like $70,000.

## Ridge Coefficients

```python
ridge = grid.best_estimator_
plt.scatter(range(X_train.shape[1]), ridge.coef_,
            c=np.sign(ridge.coef_), cmap="bwr_r")
```
.center[
![:scale 55%](images/ridge_coefficients.png)
]



Let's look at the Ridge model. This is the best estimator,
which is the model that was found in a grid search with the
best parameter settings. This looks much more reasonable.
This feature, which was a very negative one, still is very
negative. But now this is actually three and minus three. So
this is a much more reasonable range. We can also look at
the features and the effect of different values of alpha.
Here is a Ridge with 3 different values of alpha. In the
previous random seat, alpha equal to 14 was the best now and
now we have it equal to 30 something. The green one is more
or less the best setting and then there's a smaller and a
bigger one. You can see that basically what alpha does is,
on average, it pushes all the coefficients toward zero. So
here you can see this coefficient shrank going from 1 to 14
going to 0 and the same here. So basically, they all push
the different features towards 0. If you look at this long
enough, you can see things that are interesting, the first
one with alpha equal to one it's positive, and with alpha
equal to 100, it's negative. That means depending on how
much you regularize the direction of effect goes in opposite
directions, what that tells me is don't interpret your
models too much because clearly, either it has a positive or
negative effect, it can't have both.

```python
ridge100 = Ridge(alpha=100).fit(X_train, y_train)
ridge1 = Ridge(alpha=.1).fit(X_train, y_train)
plt.figure(figsize=(8, 4))

plt.plot(ridge1.coef_, 'o', label="alpha=.1")
plt.plot(ridge.coef_, 'o', label=f"alpha={ridge.alpha:.2f}")
plt.plot(ridge100.coef_, 'o', label="alpha=100")
plt.legend()
```

.center[
![:scale 60%](images/ridge_coefficients_alpha.png)
]



One other way to visualize the coefficient is to look at the
coefficient path or regularization path. On the x-axis is
the alpha, and on the y-axis, the coefficient magnitude.
Basically, I looped over all of the different alphas and you
can see how they shrink towards zero to increase alpha.
There are some very big coefficients that go to zero very
quickly and some coefficients here that stay the same for a
long time.

## Learning Curves

![:scale 90%](images/ridge_learning_curve.png)



The thing I want to illustrate is if you have less data,
regularization helps more. You can see the effects here
between the orange and the red are very drastic. The green
one probably has too much regularization. If you have very
little data, even regularizing way too much is okay, but
there's sort of a sweet spot here which is the orange one
and the orange one does best. The more data you add, the
less important regularization becomes. If you take the blue
curve at the beginning it’s much higher than the red curve,
but then, in the end, they overlap.

## Lasso Regression

$$ \min_{w \in \mathbb{R}^p, b\in\mathbb{R}} \sum_{i=1}^n (w^T\mathbf{x}_i + b - y_i)^2 + \alpha ||w||_1 $$

- Shrinks w towards zero like Ridge

- Sets some w exactly to zero - automatic feature selection!



Lasso Regression looks very similar to Ridge Regression. The
only thing that is changed is we use the L1 norm instead of
the L2 norm. L2 norm is the sum of squares, the L1 norm is
the sum of the absolute values. So again, we are shrinking w
towards 0, but we're shrinking it in a different way. The L2
norm penalizes very large coefficients more, the L1 norm
penalizes all coefficients equally. What this does in
practice is its sets some entries of W to exactly 0. It does
automatic feature selection if the coefficient of zero means
it doesn't influence the prediction and so you can just drop
it out of the model. This model does features selection
together with prediction. Ideally what you would want is,
let's say you want a model that does features selections.
The goal is to make our model automatically select the
features that are good. What you would want to penalize the
number of features that it uses, that would be L0 norm.

## Understanding L1 and L2 Penalties
.wide-left-column[
![:scale 100%](images/l2_l1_l0.png)
]

.narrow-right-column[
.smaller[
$$ \ell_2(w) = \sqrt{\sum_i w_i ^ 2}$$
$$ \ell_1(w) = \sum_i |w_i|$$
$$ \ell_0(w) = \sum_i 1_{w_i != 0}$$
]]


The L0 norm penalizes the number of features that are not
zero. The L1 norm penalizes the absolute values. L2 norm
penalizes the squared norm. The problem with L0 norm is it's
one everywhere except at zero. And that's not
differentiable, it's not even continuous, so this is very
hard to optimize. Basically, we just gave up on this and we
use the next best thing we can do, which is the L1 norm. The
L1 norm is not differentiable, there’s like this kink in
zero, but we can still optimize it pretty well. The L2 norm
penalizes large values much more than small values.

## Understanding L1 and L2 Penalties

.wide-left-column[
![:scale 100%](images/l1_kink.png)
]
.narrow-right-column[
.padding-top[
.smaller[
$ f(x) = (2 x - 1)^2 $

$ f(x) + L2 = (2 x - 1)^2 + \alpha x^2$

$ f(x) + L1= (2 x - 1)^2 + \alpha |x|$
]]]


Let's say I want to do a one-dimensional linear regression
problem. The problem to solve is something of this form. I
want my coefficients to be such that, let's say x, the one
coefficient I'm looking for and this is sort of the squared
loss function that I have. F is the daycare fitting term
here, and this is would be like some quadratic. The loss
function is in blue, its quadratic. The loss function plus
our L2 regularization, the ridge is in orange. The data
fitting plus L1 regularization is in green. The blue is a
squared quadratic function, the orange is a quadratic
function that sort of moved a little bit toward zero and the
green one also moved towards zero but there's a kink here.
Basically, this is just sort of the kink at the absolute
value of zero and that makes it much more likely that the
optimum will actually be at zero. So here the optimum for
orange is somewhere here, so it was pushed towards zero but
not exactly zero.

## Understanding L1 and L2 Penalties

![:scale 70%](images/l1l2ball.png)


A different way to visualize this is in two dimensions.
Let's say we have 2 coefficients. So one way to look at
these norms, let’s say we want to minimize the norm but say
we want to fix the norm, like more or less equivalent, these
are the L1 and L2 balls. So this is all the 2D vectors that
have Euclidean norm 1 or the ball and all the vectors that
have L1 norm is the diamond. Let's say we restrict ourselves
to a solution that lies on this ball.

## Understanding L1 and L2 Penalties

![:scale 70%](images/l1l2ball_intersect.png)


And then we have a quadratic function like this, this is
sort of the data fitting term again and so now if we want a
solution with L2 norm equal to zero, we get the solution
here, the intersection of the height lines of the data
fitting term and constraint to L2 norm. But for this
diamond, we're going to hit very likely one of the corners
just because of the geometry of the space, we're likely to
either of these corners. The corner means one of the
coefficients is exactly zero, and the other one is one. So
this is another sort of geometric realization of trying to
understand why does this lead to exact zeros hop.

## Grid-Search for Lasso
```python
param_grid = {'alpha': np.logspace(-5, 0, 10)}
grid = GridSearchCV(Lasso(normalize=True), param_grid, cv=10)
grid.fit(X_train, y_train)

print(grid.best_params_)
print(grid.best_score_)
```
```
{'alpha': 0.0016}
0.163
```




Now we can do Grid Search again, the default parameters
usually don't work very well for Lasso. I use alpha on the
logarithmic grid. I fitted and then I get the best score.

![:scale 90%](images/lasso_alpha_triazine.png)



Looking at the training test set performance, you can see
that if you increase the regularization, this model gets
really bad. The ridge regression didn't go this badly. If
you set the realization to one, all coefficients become
zero. Other than that there's reasonable performance, which
is about as good as the ridge performance.

.center[
![:scale 60%](images/lasso_coefficients.png)
]

```python
print(X_train.shape)
np.sum(lasso.coef_ != 0)
```
```
(139, 60)
13
```



These are the coefficients of the model. Out of the 104
features, it only selected 64 that are non-zero, the other
ones are exactly zero. You can see this visualized here. The
white ones are exactly zero and the other ones on non-zero.
If I wanted I could prune the future space a lot and that
makes the model possibly more interpretable. There's a
slight caveat here if two of the features that are very
correlated, Lasso will pick one of them at random and make
the other one zero. Just because something's zero doesn't
mean it's not important. It means you can drop it out of
this model. If you have two features that are identical, one
of them will be zero and one of them will be not zero and
it's going to be randomly selected. That makes
interpretation a little bit harder.

## Elastic Net

- Combines benefits of Ridge and Lasso

- two parameters to tune.


$$\min_{w \in \mathbb{R}^p, b\in\mathbb{R}} \sum_{i=1}^n ||w^T\mathbf{x}_i + b - y_i||^2 + \alpha_1 ||w||_1 +  \alpha_2 ||w||^2_2 $$



You can also combine them. This actually what works best in
practice. This is what's called the Elastic Net. Elastic Net
tries to combine both of these penalizations together. You
now have both terms, you have the L1 norm and the L2 norm
and you have different values of alpha. Basically, this
generalizes both. If you choose both these are alpha, it can
become ridge and it can become Lasso, it can become any
anything in between. Generally, ridge helps generalization.
So it's a good idea to have the ridge penalty in there, but
also maybe if there are some features that are really not
useful, the L1 penalty helps makes the same exactly zero.

## Comparing unit balls

![:scale 70%](images/l1l2_elasticnet.png)



Looking at the unit balls, if you zoom in here you would see
the Elastic Net ball is kind of round, but also has a
corner. Important things are the corners because that allows
you to hit the exact zeros.

## Parametrization in scikit-learn
$$\min_{w \in \mathbb{R}^p, b\in\mathbb{R}} \sum_{i=1}^n (w^T\mathbf{x}_i + b - y_i)^2 + \alpha \eta ||w||_1 +  \alpha (1 - \eta) ||w||^2_2 $$

Where $\eta$ is the relative amount of l1 penalty (`l1_ratio` in the code).


The way this parameterize in scikit-learn is slightly
different. In scikit-learn, you have a parameter alpha,
which is the amount of regularization and then there's a
parameter called l1_ratio, that says how much of the penalty
should be L1 and L2. If you make this one, you have Lasso,
if you make it zero, you have a Ridge. Don't use Lasso or
Ridge and set alpha zero, because the solver will not handle
it well. If you actually want alpha equal to zero, use
linear regression. Now we have more parameters to tune, but
we just have a more general model. This actually works
pretty well often.

## Grid-searching ElasticNet

```python
from sklearn.linear_model import ElasticNet
param_grid = {'alpha': np.logspace(-4, -1, 10),
              'l1_ratio': [0.01, .1, .5, .8, .9, .95, .98, 1]}

grid = GridSearchCV(ElasticNet(), param_grid, cv=10)
grid.fit(X_train, y_train)

print(grid.best_params_)
print(grid.best_score_)
```
```
{'alpha': 0.001, 'l1_ratio': 0.9}
0.100
```
```python
(grid.best_estimator_.coef_!= 0).sum()
```
```
10

```


Here is me doing a grid search. If you have two parameters
for the grid search it will do all possible combinations.
Here I do a logarithmic space for alpha and for the
l1_ratio, I use something that’s very close to zero and
something that's very close to one and some stuff in
between. If you want to analyze the output of a 2D grid
search a little bit harder we can’t do the nice curve
anymore.

## Analyzing grid-search results
```python
import pandas as pd
res = pd.pivot_table(pd.DataFrame(grid.cv_results_),
    values='mean_test_score', index='param_alpha', columns='param_l1_ratio')
```
.center[
![:scale 60%](images/elasticnet_search.png)
]


The way that I like to do it is, here's the grip.cv results.
And I put it in a data frame and then I'll make a pivot
table where the values are test score, the index is one
parameter and the columns are the other parameter. It allows
me to visualize the grid search nicely. This is alpha and
this is l1_ratio and you can see that if the l1_ratio is
pretty high, there are some pretty good results, if you set
the alpha accordingly. So here's like the diagonal of pretty
good things. This is the model that did best. There's a
slight caveat here that right now I did this with
cross-validation and so this is the cross-validation
accuracy. Last time I said, this is not really a good
measure of generalization performance. So here, I searched
way more parameters, I tried like 5 or 10 times as many
models. So it's likely that by chance, I'll get better
results. I didn't do this here in particular, because the
data set is small and very noisy but in practice, if you
want to compare models, you should evaluate it on a test set
and see which of the models actually are better on the test.
One more thing, why this is helpful is if the best value is
on the edge of this graph that means my ranges were too
small. Question is why we're using r square instead of the
squared loss, one of the answers is that's the default in
scikit-learn and the other answer is it's nice to know the
range so you know that perfect prediction is one and you
have some idea of what 0.5 means, the RMSE (the other norm
that you usually use is the RMSE) depends on the scale of
the output. So for example for the housing prices, it might
be interesting to see what is the standard error in terms of
dollars. If you want, like something that is in the units of
the output, RMSE is good or mean absolute error might even
be better. If you want something that is independent of the
units of the output r square is pretty good because you know
it's going to be between zero and one and it's measure
something like the correlation and so if it's like 0.9, you
know it’s a pretty good model. If my RMSE is 10,000 I don't
know if have a good model or a bad model depends on what the
range of the outputs is. The last thing I want to talk about
today is this was basically changing the regularization
parts. The two most times regularization we looked at is
Ridge which is L2 penalty, Lasso which is an L1 penalty and
combining two of them which is Elastic Net. So now I want to
talk about changing the first part, which was the squared
loss of the predictions, basically.


## Questions ?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
sklearn.set_config(print_changed_only=True)

In [ ]:
from sklearn.linear_model import Ridge, LinearRegression

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
X, y = boston.data, boston.target

In [ ]:
print(boston.DESCR)

In [ ]:
X.shape

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(20, 10))
for i, ax in enumerate(axes.ravel()):
    if i > 12:
        ax.set_visible(False)
        continue
    ax.plot(X[:, i], y, 'o', alpha=.5)
    ax.set_title("{}: {}".format(i, boston.feature_names[i]))
    ax.set_ylabel("MEDV")

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

In [ ]:
np.mean(cross_val_score(LinearRegression(),
                        X_train, y_train, cv=10))

In [ ]:
np.mean(cross_val_score(
        Ridge(), X_train, y_train, cv=10))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha': np.logspace(-3, 3, 14)}
print(param_grid)

In [ ]:
grid = GridSearchCV(Ridge(), param_grid, cv=10, return_train_score=True)
grid.fit(X_train, y_train)

In [ ]:
import pandas as pd
plt.figure(dpi=200)
results = pd.DataFrame(grid.cv_results_)
results.plot('param_alpha', 'mean_train_score', ax=plt.gca())
results.plot('param_alpha', 'mean_test_score', ax=plt.gca())

plt.legend()
plt.xscale("log")

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, scale
# being lazy and not really doing things properly whoops
X_poly = PolynomialFeatures(include_bias=False).fit_transform(scale(X))
print(X_poly.shape)
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, random_state=42)

In [ ]:
np.mean(cross_val_score(LinearRegression(),
                        X_train, y_train, cv=10))

In [ ]:
np.mean(cross_val_score(Ridge(),
                        X_train, y_train, cv=10))

In [ ]:
grid = GridSearchCV(Ridge(), param_grid, cv=10, return_train_score=True)
grid.fit(X_train, y_train)

In [ ]:
results = pd.DataFrame(grid.cv_results_)

results.plot('param_alpha', 'mean_train_score', ax=plt.gca())
results.plot('param_alpha', 'mean_test_score', ax=plt.gca())
plt.legend()
plt.xscale("log")

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
lr = LinearRegression().fit(X_train, y_train)
plt.scatter(range(X_poly.shape[1]), lr.coef_, c=np.sign(lr.coef_), cmap="bwr_r")

In [ ]:
ridge = grid.best_estimator_
plt.scatter(range(X_poly.shape[1]), ridge.coef_, c=np.sign(ridge.coef_), cmap="bwr_r")

In [ ]:
ridge100 = Ridge(alpha=100).fit(X_train, y_train)
ridge1 = Ridge(alpha=1).fit(X_train, y_train)
plt.figure(figsize=(8, 4))

plt.plot(ridge1.coef_, 'o', label="alpha=1")
plt.plot(ridge.coef_, 'o', label="alpha=14")
plt.plot(ridge100.coef_, 'o', label="alpha=100")
plt.legend()

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso().fit(X_train, y_train)
print("Training set score: {:.2f}".format(lasso.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lasso.score(X_test, y_test)))
print("Number of features used:", np.sum(lasso.coef_ != 0))

# Exercise
Load the diabetes dataset using ``sklearn.datasets.load_diabetes``. Apply ``LinearRegression``, ``Ridge`` and ``Lasso`` and visualize the coefficients. Try polynomial features.

In [ ]:
# %load solutions/linear_models_diabetes.py